# Inverting Matrices with Spiking Neurons

Suppose we'd like to build a Spiking Neural Network (SNN) that can invert some input
matrix, purely by using spiking nonlinearities. It is not at all obvious how to do
so using existing software libraries. Gyrus makes this surprisingly simple, accurate,
and scalable, once the problem is approached from the perspective of expressing the
solution as a suitable algorithm in NumPy.

In [ ]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf
from matplotlib.colors import Normalize
from nengo.utils.numpy import rms
from nengo_gui.jupyter import InlineGUI

import gyrus

In [ ]:
rng = np.random.RandomState(seed=0)
A = rng.randn(2, 2)  # test case
print("Condition number:", np.linalg.cond(A))

## Nengo Approach

Even in the 1x1 case this task is quite challenging, as it reduces to trying to
approximate $f(x) = 1 / x$, which is an unnatural function to fit using the tuning
curves of a Nengo ensemble.

Nevertheless, we may attempt this in Nengo by decoding the matrix inverse from an
ensemble, like so:

In [ ]:
# Only use evaluation points with small condition numbers prevent the decoders
# from trying to fit arbitrarily large inverses.
max_cond = 10
eval_points = []
for Atrain in rng.randn(5000, *A.shape):
    if np.linalg.cond(Atrain) < max_cond:
        eval_points.append(Atrain.flatten())
radius = np.percentile(np.linalg.norm(eval_points, axis=-1), q=90)


def function(x):
    if np.all(x == 0):
        return x
    return np.linalg.inv(x.reshape(A.shape)).flatten()


with nengo.Network(seed=0) as model:
    stim = nengo.Node(A.flatten())

    x = nengo.Ensemble(
        n_neurons=2500,
        dimensions=A.size,
        eval_points=eval_points,
        radius=radius,
    )

    Ainv_hat = nengo.Node(size_in=A.size)

    nengo.Connection(stim, x, synapse=None)
    nengo.Connection(x, Ainv_hat, function=function, synapse=None)

    p = nengo.Probe(Ainv_hat, synapse=0.2)

with nengo.Simulator(model) as sim:
    sim.run(2)

In [ ]:
def evaluate(A, Ainv_hat):
    Ainv = np.linalg.inv(A)
    nrmse = rms(Ainv - Ainv_hat) / rms(Ainv)
    print(f"Normalized RMSE: {(100 * nrmse).round(1)}%")

    vlim = np.max(np.abs([Ainv, Ainv_hat]))
    cmap = cm.get_cmap("bwr")
    im = cm.ScalarMappable(cmap=cmap, norm=Normalize(-vlim, vlim))

    fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)
    axes[0].set_title("Ground Truth ($A^{-1}$)")
    axes[0].imshow(Ainv, vmin=-vlim, vmax=vlim, cmap=cmap)
    axes[1].set_title(r"Approximation ($\tilde{A}^{-1}$)")
    axes[1].imshow(Ainv_hat, vmin=-vlim, vmax=vlim, cmap=cmap)
    axes[2].set_title(r"Error ($A^{-1} - \tilde{A}^{-1}$)")
    axes[2].imshow(Ainv - Ainv_hat, vmin=-vlim, vmax=vlim, cmap=cmap)
    fig.colorbar(im, ax=axes.ravel().tolist())
    fig.show()


def plot_simulation(A, data):
    data = np.asarray(data).reshape(-1, A.size)
    Ainv = np.linalg.inv(A).flatten()
    plt.figure()
    for i in range(A.size):
        (line,) = plt.plot(data[:, i])
        plt.hlines(Ainv[i], 0, len(data) - 1, color=line.get_color(), linestyle="--")
    plt.xlabel("Time-step")
    plt.show()


plot_simulation(A, sim.data[p])
evaluate(A, sim.data[p][-1].reshape(A.shape))

The results look okay visually. But even with 2,500 neurons and fiddling with the
``eval_points`` in order to get it to properly train, the normalized root mean-squared
error (NRMSE) is around 20%. And it is not clear how to efficiently scale this up to
larger matrices (try this again with a 3x3 matrix and see) or how to improve the
accuracy.

One might try to use NengoDL to backpropagate across a deeper network. But a simpler
solution, that does not require any backpropagation at all, is to approach this as an
iterative optimization problem, which we can do using NumPy (and thus Gyrus as well).
As we will see, this algorithm is converted into a recurrent SNN that is cheap
to train and scales efficiently to larger matrices.

## NumPy Approach

There are a few approaches to implementing matrix inversion as an algorithm using basic
NumPy operations. One that fits very naturally with SNNs is to start with some initial
guess of the inverse (e.g., all zeros), and then iteratively update it in the direction
that minimizes the error. This is also known as gradient descent. In the simplest case,
the iterative updates are Euler method updates of the gradient with respect to the
mean-squared error in the current estimate.

Specifically, if $M$ is the current guess of $A^{-1}$, then we would like for $MA = I$.
Thus, the error that we wish to minimize is the mean-squared error, $\sum (MA - I)^2$.
Taking the gradient of this with respect to $M$ is $2 (MA - I)A^T$, which may look
familiar from least-squares optimization. The NumPy algorithm is then to simply
integrate this gradient, as follows.

In [ ]:
def gradient(A, M):
    """Compute the gradient of M approximating inv(A)."""
    I = np.eye(A.shape[0])
    return 2 * (M @ A - I) @ A.T


def numpy_inverse(A, M, dt, n_iter, verbose=False):
    """Compute the inverse of A by gradient descent from M."""
    data = []
    for _ in range(n_iter):
        if verbose:
            data.append(M.copy())
        M -= dt * gradient(A, M)

    if verbose:
        plot_simulation(A, data)

    return M


Ainv_hat = numpy_inverse(A=A, M=np.zeros_like(A), dt=0.05, n_iter=50, verbose=True)
evaluate(A, Ainv_hat)

This gets us the correct answer (i.e., approximately 0.0% normalized RMSE). However,
this is not using spiking neurons. Nevertheless, it is using primitive NumPy
operations that we can automagically convert to Nengo using Gyrus.

## Gyrus Approach - Initial

The `gradient` function is already compatible with Gyrus without any modification. And
so is the `numpy_inverse` function in fact! But, the `for` loop will essentially create
`n_iter=50` layers of the exact same network. We can use the code as is, but it will be
slow and not scale well to larger matrices or more iterations.

The solution is to recognize that the `gyrus.integrate` operation is like a `for` loop;
this operation rolls up some additive update into a recurrent network by integrating
some given `integrand` over time. In this case, the integrand is just `-gradient(A, M)`
with appropriate scaling by Nengo's default time-step of `1e-3`. Since we're using
spiking neurons, we also filter the integral with a lowpass synapse to decode the final
approximation.

In [ ]:
def gyrus_inverse(A, M, dt, synapse=None):
    """Compute the inverse of A by gradient descent from M."""
    return M.integrate_fold(
        integrand=lambda M: -dt * gradient(A, M) / 1e-3,
        synapse=synapse,
    )

In [ ]:
op = gyrus_inverse(
    A=gyrus.stimuli(A).configure(
        n_neurons=1000,
        input_magnitude=2,
        seed=0,
    ),
    M=gyrus.stimuli(np.zeros_like(A)),
    dt=0.05,
).filter(0.2)

In [ ]:
out = np.asarray(op.run(1)).squeeze(axis=-1)
Ainv_hat = out[..., -1]

plot_simulation(A, np.moveaxis(out, -1, 0))  # move time to first dimension
evaluate(A, Ainv_hat)

This works quite a bit better, and all nonlinearities are implemented using spiking LIF
neurons! Moreover, relatively little effort was required to build and try this out (only
a few extra lines of code).

Now let's see what this network looks like in the NengoGUI.

In [ ]:
with nengo.Network() as model:
    op.make()

InlineGUI(model)

There's a lot going on there, but it's all just multiply operators (i.e., Nengo product
networks) connected together with linear transforms in a very complicated but specific
way. If one were to try to recreate this without using Gyrus, that is, by specifying
each primitive Nengo object one statement at a time, the endeavour would not only be
arduous, but the result would be difficult to understand and modify.

## Gyrus Approach - Optimizing Spikes

We can improve the efficiency of the SNN in terms of how many spikes are needed to
obtain some level of accuracy by applying a number of advanced techniques:

1. For each ensemble, use just a single `gyrus.Parabola()` response curve centered
around 0, which is ideal for the Product network.
2. Use the `nengo.solvers.Lstsq()` solver since regularization is no longer required
with the above.
3. To integrate the gradient over time, use the Keras `Adam` optimizer as a synapse
(`gyrus.Adam`), instead of Euler's method. This also make the network significantly more
robust to changes in hyperparameters.

We demonstrate that this can scale by inverting a 5x5 matrix, and count the number of
spikes required to achieve a given normalized RMSE.

In [ ]:
def go(A, dt, t, rates, synapse, output_synapse):
    op = gyrus_inverse(
        A=gyrus.stimuli(A).configure(n_neurons=1),
        M=gyrus.stimuli(np.zeros_like(A)),
        dt=dt,
        synapse=synapse,
    ).filter(output_synapse)

    with nengo.Network() as model:
        # RegularSpiking wrapper requires nengo>=3.1.0.
        model.config[nengo.Ensemble].neuron_type = nengo.RegularSpiking(
            gyrus.Parabola()
        )
        model.config[nengo.Ensemble].encoders = nengo.dists.Choice([[1]])
        model.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
        model.config[nengo.Ensemble].max_rates = nengo.dists.Choice([rates])
        model.config[nengo.Connection].solver = nengo.solvers.Lstsq()
        accessor = gyrus.probe(op.make())

        # Count spikes to analyze efficiency.
        spike_counter = nengo.Node(size_in=1)
        for ens in model.all_ensembles:
            # Note: Spikes are scaled by 1 / dt in Nengo.
            nengo.Connection(ens.neurons, spike_counter, transform=1e-3, synapse=None)
        p_spike_count = nengo.Probe(spike_counter)

    with tf.device("/cpu:0"):
        with nengo.Simulator(model, optimize=False) as sim:
            sim.run(t)

    return model, accessor(sim), sim.data[p_spike_count]

In [ ]:
rng = np.random.RandomState(seed=0)
A = rng.randn(5, 5)  # test case
print("Condition number:", np.linalg.cond(A))

In [ ]:
model, data, spike_counts = go(
    A=A,
    dt=0.1,
    t=2.5,
    rates=1e3,
    synapse=gyrus.Adam(1),
    output_synapse=0.2,
)
out = np.asarray(data).squeeze(axis=-1)

In [ ]:
Ainv_hats = np.moveaxis(out, -1, 0)  # move time to first dimension
Ainv_hat = out[..., -1]

plot_simulation(A, Ainv_hats)
evaluate(A, Ainv_hat)

In [ ]:
Ainv = np.linalg.inv(A)
nrmse = 100 * rms(Ainv[None, :] - Ainv_hats, axis=(1, 2)) / rms(Ainv)

plt.figure()
plt.plot(np.cumsum(spike_counts), nrmse)
# plt.yscale("log")
plt.xlabel("Total # Spikes")
plt.ylabel("Normalized RMSE (%)")
plt.show()

print(f"# Neurons: {model.n_neurons}")

In [ ]:
InlineGUI(model)

The model has 500 parabolic neurons in total, and inverts a 5x5 matrix with <0.5% NRMSE
after generating on the order of a million spikes.

Resources can be parallelized by increasing `n_neurons` and decreasing the `rates` of
each neuron. Also, the NRMSE decreases as a function of the total number of spikes being
generated (e.g., ~10% NRMSE is achieved after roughly half as many spikes are
generated). Thus, we can dynamically trade between latency, energy, and precision, by
dialing certain model parameters.

In general, the ideal model settings will depend on the hardware, its supported neuron
response curves, the energy budget, the timing requirements of an application, and the
level of accuracy required downstream. These trade-offs are discussed in more detail in
[A spike in performance: Training hybrid-spiking neural networks with quantized
activation functions](https://arxiv.org/abs/2002.03553) and [Dynamical Systems in
Spiking Neuromorphic Hardware](https://uwspace.uwaterloo.ca/handle/10012/14625).